In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from time import time
import datetime
import gc
pd.set_option('display.max_columns',100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [ ]:
metadata_dtype = {'site_id':"uint8",'building_id':'uint16','square_feet':'uint32','year_built':'float32','floor_count':'float32'}
metadata = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv",dtype=metadata_dtype)
metadata.info(memory_usage='deep') 
#memory_usage, deep 일 경우 위 오브젝트 dtype을 조사하여 데이터를 자세하게 조사하고 리턴된 값에 포함

In [ ]:
weather_dtype = {"site_id":"uint8",'air_temperature':"float32",'cloud_coverage':"float32",'dew_temperature':"float32",'precip_depth_1_hr':"float32",
                 'sea_level_pressure':"float32",'wind_direction':"float32",'wind_speed':"float32"}

weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv",parse_dates=['timestamp'],dtype=weather_dtype)
weather_test = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv",parse_dates=['timestamp'],dtype=weather_dtype)
print (weather_train.info(memory_usage='deep'))
print ("-------------------------------------")
print (weather_test.info(memory_usage='deep'))

In [ ]:
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv",parse_dates=['timestamp'])
#parse_dates 는 날짜시간으로 시작된 컬럼을 datatime으로 parsing
test_cols_to_read = ['building_id','meter','timestamp']
test = pd.read_csv("../input/ashrae-energy-prediction/test.csv",parse_dates=['timestamp'],usecols=test_cols_to_read)
#pars_cols는 한번 사용 후 더이상 사용x, 대신 usecols사용

In [ ]:
test_cols_to_read

In [ ]:
#weather_train/test 데이터셋 feature 별 결측치 확인
missing_weather = pd.DataFrame(weather_train.isna().sum()/len(weather_train),columns=["Weather_Train_Missing_Pct"])
missing_weather["Weather_Test_Missing_Pct"] = weather_test.isna().sum()/len(weather_test)
missing_weather

In [ ]:
print(metadata.count()) #실제 데이터 수치
print()
print(metadata.isna().sum()) #데이터 결측치
print()
print(metadata.isna().sum()/len(metadata)*100) #데이터 결측치 퍼센트로 나옴
print()
print(metadata.isna().mean()) #mean() 함수로 써도 나타낼 수 있다.

In [ ]:
missing_train_test = pd.DataFrame(train.isna().sum()/len(train),columns=["Missiong_Pct_Train"])
missing_train_test["Missing_Pct_Test"] = test.isna().sum()/len(test)
missing_train_test

In [ ]:
print("train.head()")
print(train.head())
print()
print("train.describe()")
print(train.describe(include='all')) #include all을하면 list의 dtypes 또는 none 다 출력
print()


In [ ]:
#train['meter'].replace({0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"},inplace=True)
#test['meter'].replace({0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"},inplace=True)

In [ ]:
sns.countplot(train['meter'])
plt.title("Distribution of meter")
plt.xlabel("meter ID")
plt.ylabel("count")

In [ ]:
train['building_id'].nunique() #유니크한 값의 총 개수를 표현

In [ ]:
train[train['building_id'] == 1094]['meter'].unique()

In [ ]:
train.groupby('meter')['meter_reading'].describe()#agg(['min','max','mean','median','count','std'])
#groupby 함수를 사용할경우 'meter'컬럼의 train의 모든열에 대해 통계량이 계산됨
#agg 를 사용할 경우 일반적인 통계함수를 적용, describe()는 전체 통계량
#std는 표준편차임, 평균에서 관측치가 얼마나 떨어져있나 표시

In [ ]:
for df in [train, test]:
    df['Month'] = df['timestamp'].dt.month.astype("uint8")
    df['DayOfMonth'] = df['timestamp'].dt.day.astype("uint8")
    df['DayOfWeek'] = df['timestamp'].dt.dayofweek.astype("uint8")
    df['Hour'] = df['timestamp'].dt.hour.astype("uint8")

In [ ]:
#train.groupby(['meter','Month','DayOfMonth','DayOfWeek','Hour'])['meter_reading'].agg(['max','mean','median','count','std'])
train.groupby(['meter','Month'])['meter_reading'].agg(['max','mean','median','count','std'])


In [ ]:
train.groupby(['meter','DayOfWeek'])['meter_reading'].agg(['max','mean','median','count','std'])

In [ ]:
train.head(20)

In [ ]:
train['meter_reading'].describe()

In [ ]:
sns.distplot(np.log1p(train['meter_reading']),kde=True)
#kde는 커널밀도곡선으로 같이 표시하려면 True
#np.log1p(z) = log(1+z) 
plt.title("Distribution of Log of Meter Reading Variable")
# Lot of 0 values as can be seen from the distribution

In [ ]:
sns.distplot(train['meter_reading'],kde=True)
#kde는 커널밀도곡선으로 같이 표시하려면 True
#np.log1p(z) = log(1+z) 
plt.title("Distribution of Log of Meter Reading Variable")
# Lot of 0 values as can be seen from the distribution

In [ ]:
sns.boxplot(train[train['meter'] == 0]['meter_reading'])
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Boxplot of Meter Reading Variable for the Meter Type: 0(Electricity)")

In [ ]:
sns.boxplot(train[train['meter'] == 1]['meter_reading'])
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Boxplot of Meter Reading Variable for the Meter Type: 1(ChilledWater)")

In [ ]:
sns.boxplot(train[train['meter'] == 2]['meter_reading'])
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Boxplot of Meter Reading Variable for the Meter Type: 2(Steam)")

In [ ]:
sns.boxplot(train[train['meter'] == 3]['meter_reading'])
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Boxplot of Meter Reading Variable for the Meter Type: 3(HotWater)")

In [ ]:
#값 스케일링 변경
train['meter_reading'] = np.log1p(train['meter_reading'])

In [ ]:
sns.distplot(train[train['meter'] == 0]['meter_reading'],kde=False)
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Distribution of Meter Reading Variable for the Meter Type: 0(Electricity)")

In [ ]:
sns.distplot(train[train['meter'] == 1]['meter_reading'],kde=False)
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Distribution of Meter Reading Variable for the Meter Type: 1:ChilledWater")

In [ ]:
sns.distplot(train[train['meter'] == 2]['meter_reading'],kde=False)
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Distribution of Meter Reading Variable for the Meter Type: 2:Steam")

In [ ]:
sns.distplot(train[train['meter'] == 3]['meter_reading'],kde=False)
# 0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"
plt.title("Distribution of Meter Reading Variable for the Meter Type: 3:HotWater")

In [ ]:
train.head()

In [ ]:
metadata.info()

In [ ]:
metadata.head()

In [ ]:
cols = ['site_id','primary_use','year_built','floor_count','building_id']
for col in metadata:
    print ("Number of Unique Values in the {} column are:".format(col),metadata[col].nunique())

In [ ]:
cols = ['site_id','primary_use','year_built','floor_count']
for col in cols:
    print ("Unique Values in the {} column are:".format(col),metadata[col].unique())
    print ("\n")

In [ ]:
sns.countplot(metadata['site_id'])
plt.title("Count of Site_id in the Metadata table")
plt.xlabel("Site_Id")
plt.ylabel("Count")

In [ ]:
plt.figure(figsize=(8,6))
metadata['primary_use'].value_counts().sort_values().plot(kind='bar')
plt.title("Count of Primary_Use Variable in the Metadata table")
plt.xlabel("Primary Use")
plt.ylabel("Count")
plt.xticks(rotation=90) #x축 인덱스 각도

In [ ]:
metadata['square_feet'].describe()

In [ ]:
sns.boxplot(metadata['square_feet'])

In [ ]:
metadata['square_feet'] = np.log1p(metadata['square_feet'])

In [ ]:
sns.distplot(metadata['square_feet'])
plt.title("Distribution of Square Feet variable of Metadata Table")
plt.xlabel("Area in Square Feet")
plt.ylabel("Frequency")
# Looks like a normal distribution distribution

In [ ]:
sns.boxplot(metadata['square_feet'])
plt.title("Box Plot of Square Feet Variable")
# There are a few outliers visible

In [ ]:
metadata.groupby('primary_use')['square_feet'].agg(['mean','median','count','std']).sort_values(by='count')
# Parking has the highest average are although the count is less.
# Education has the highest count as can be seen in the countplot above.

In [ ]:
metadata.groupby('site_id')['square_feet'].agg(['mean','median','count','std']).sort_values(by='count')
# Parking has the highest average are although the count is less.
# Education has the highest count as can be seen in the countplot above.

In [ ]:
metadata['year_built'].value_counts().sort_values().plot(kind='bar',figsize=(15,6))
#metadata['year_built'].value_counts().plot(kind='bar',figsize=(15,6))
plt.xlabel("Year Built")
plt.ylabel("Count")
plt.title("Distribution of Year Built Variable")

In [ ]:
sns.countplot(metadata['year_built'])
plt.xlabel("Year Built")
plt.ylabel("Count")
plt.title("Distribution of Year Built Variable")
plt.xticks(rotation=90)

In [ ]:
metadata['floor_count'].value_counts(dropna=False).sort_index().plot(kind='bar',figsize=(8,6))
plt.xlabel("Number of Floors")
plt.ylabel("Count of Buildings")
# Lot of missing values here as well
# Maximum number of floors is 26

In [ ]:
metadata.groupby('floor_count')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata.groupby('primary_use')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata.groupby('primary_use')['floor_count'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata.head()

In [ ]:
#fillna 함수는 NaN 값을 다른것으로 대체
metadata['year_built'].fillna(-999, inplace=True)
#NaN 값을 -999로 변경 및 실제 데이터프레임에 저장(inplace=true)
metadata['year_built'] = metadata['year_built'].astype('int16')
metadata['floor_count'].fillna(-999, inplace=True)
metadata['floor_count'] = metadata['floor_count'].astype('float32')

In [ ]:
metadata.head()

**Weather Data(Train)**

In [ ]:
weather_train.head()

In [ ]:
weather_train.info(memory_usage='deep')

In [ ]:
#결측치 있는 column 에 대해 조회
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    print (" Minimum Value of {} column is {}".format(col,weather_train[col].min()))
    print (" Maximum Value of {} column is {}".format(col,weather_train[col].max()))
    print ("----------------------------------------------------------------------")

In [ ]:
weather_train.isna().mean()

In [ ]:
weather_train[['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']].describe()

1. Missing values in air_temperature, cloud_coverage, dew_temperature, precip_depth_1_hr, sea_level_pressure, wind_speed variables
2. There are negative values in air_temperature, dew_temperature and precip_depth_1_hr variables.
3. Looks like there are outliers in precip_depth_1_hr variable (can be guessed from Max value).
4. min value of wind_speed as 0 does not make any sense.

In [ ]:
weather_train['timestamp'].describe()

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.distplot(weather_train[col].dropna())
    #결측치를 제외하고 각각 값 출력

1. Distribution of sea_level_pressure looks like a normal distribution.
2. Lot of 0 values in precip_depth_1_hr variable.
3. Wind_Speed distribution looks like positively skewed.
4. Dew Temperature looks like a Negatively skewed distribution.
5. Cloud_Coverage takes distinct values unlike these other variables.

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.boxplot(weather_train[col].dropna())

**Weather_Test data**

In [ ]:
weather_test.info(memory_usage='deep')

In [ ]:
weather_test['timestamp'].describe()
# The time duration is similar to the test dataset.

**Merging Data**

In [ ]:
%%time
train = pd.merge(train,metadata,on='building_id',how='left')
test  = pd.merge(test,metadata,on='building_id',how='left')
print ("Training Data Shape {}".format(train.shape))
print ("Testing Data Shape {}".format(test.shape))
gc.collect()

In [ ]:
%%time
train = pd.merge(train,weather_train,on=['site_id','timestamp'],how='left')
test  = pd.merge(test,weather_test,on=['site_id','timestamp'],how='left')
print ("Training Data Shape {}".format(train.shape))
print ("Testing Data Shape {}".format(test.shape))
gc.collect()

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    train[col].fillna(train[col].mean(),inplace=True)
    test[col].fillna(test[col].mean(),inplace=True)
    
del metadata, weather_train, weather_test

In [ ]:
%%time
for df in [train, test]:
    df['air_temperature'] = df['air_temperature'].astype('float16')
    df['cloud_coverage'] = df['cloud_coverage'].astype("float16")
    df['dew_temperature'] = df['dew_temperature'].astype('float16')
    df['precip_depth_1_hr'] = df['precip_depth_1_hr'].astype('float32')
    df['sea_level_pressure'] = df['sea_level_pressure'].astype('float32')
    df['wind_direction'] = df['wind_direction'].astype('float32')
    df['wind_speed'] = df['wind_speed'].astype('float16')
    df['square_feet'] = df['square_feet'].astype("float32")
    df['building_id'] = df['building_id'].astype("int16")

In [ ]:
train.drop('timestamp',axis=1,inplace=True)
test.drop('timestamp',axis=1,inplace=True)

In [ ]:
%%time
number_unique_meter_per_building = train.groupby('building_id')['meter'].nunique()
train['number_unique_meter_per_building'] = train['building_id'].map(number_unique_meter_per_building)


mean_meter_reading_per_building = train.groupby('building_id')['meter_reading'].mean()
train['mean_meter_reading_per_building'] = train['building_id'].map(mean_meter_reading_per_building)
median_meter_reading_per_building = train.groupby('building_id')['meter_reading'].median()
train['median_meter_reading_per_building'] = train['building_id'].map(median_meter_reading_per_building)
std_meter_reading_per_building = train.groupby('building_id')['meter_reading'].std()
train['std_meter_reading_per_building'] = train['building_id'].map(std_meter_reading_per_building)


mean_meter_reading_on_year_built = train.groupby('year_built')['meter_reading'].mean()
train['mean_meter_reading_on_year_built'] = train['year_built'].map(mean_meter_reading_on_year_built)
median_meter_reading_on_year_built = train.groupby('year_built')['meter_reading'].median()
train['median_meter_reading_on_year_built'] = train['year_built'].map(median_meter_reading_on_year_built)
std_meter_reading_on_year_built = train.groupby('year_built')['meter_reading'].std()
train['std_meter_reading_on_year_built'] = train['year_built'].map(std_meter_reading_on_year_built)


mean_meter_reading_per_meter = train.groupby('meter')['meter_reading'].mean()
train['mean_meter_reading_per_meter'] = train['meter'].map(mean_meter_reading_per_meter)
median_meter_reading_per_meter = train.groupby('meter')['meter_reading'].median()
train['median_meter_reading_per_meter'] = train['meter'].map(median_meter_reading_per_meter)
std_meter_reading_per_meter = train.groupby('meter')['meter_reading'].std()
train['std_meter_reading_per_meter'] = train['meter'].map(std_meter_reading_per_meter)


mean_meter_reading_per_primary_usage = train.groupby('primary_use')['meter_reading'].mean()
train['mean_meter_reading_per_primary_usage'] = train['primary_use'].map(mean_meter_reading_per_primary_usage)
median_meter_reading_per_primary_usage = train.groupby('primary_use')['meter_reading'].median()
train['median_meter_reading_per_primary_usage'] = train['primary_use'].map(median_meter_reading_per_primary_usage)
std_meter_reading_per_primary_usage = train.groupby('primary_use')['meter_reading'].std()
train['std_meter_reading_per_primary_usage'] = train['primary_use'].map(std_meter_reading_per_primary_usage)


mean_meter_reading_per_site_id = train.groupby('site_id')['meter_reading'].mean()
train['mean_meter_reading_per_site_id'] = train['site_id'].map(mean_meter_reading_per_site_id)
median_meter_reading_per_site_id = train.groupby('site_id')['meter_reading'].median()
train['median_meter_reading_per_site_id'] = train['site_id'].map(median_meter_reading_per_site_id)
std_meter_reading_per_site_id = train.groupby('site_id')['meter_reading'].std()
train['std_meter_reading_per_site_id'] = train['site_id'].map(std_meter_reading_per_site_id)


test['number_unique_meter_per_building'] = test['building_id'].map(number_unique_meter_per_building)

test['mean_meter_reading_per_building'] = test['building_id'].map(mean_meter_reading_per_building)
test['median_meter_reading_per_building'] = test['building_id'].map(median_meter_reading_per_building)
test['std_meter_reading_per_building'] = test['building_id'].map(std_meter_reading_per_building)

test['mean_meter_reading_on_year_built'] = test['year_built'].map(mean_meter_reading_on_year_built)
test['median_meter_reading_on_year_built'] = test['year_built'].map(median_meter_reading_on_year_built)
test['std_meter_reading_on_year_built'] = test['year_built'].map(std_meter_reading_on_year_built)

test['mean_meter_reading_per_meter'] = test['meter'].map(mean_meter_reading_per_meter)
test['median_meter_reading_per_meter'] = test['meter'].map(median_meter_reading_per_meter)
test['std_meter_reading_per_meter'] = test['meter'].map(std_meter_reading_per_meter)

test['mean_meter_reading_per_primary_usage'] = test['primary_use'].map(mean_meter_reading_per_primary_usage)
test['median_meter_reading_per_primary_usage'] = test['primary_use'].map(median_meter_reading_per_primary_usage)
test['std_meter_reading_per_primary_usage'] = test['primary_use'].map(std_meter_reading_per_primary_usage)

test['mean_meter_reading_per_site_id'] = test['site_id'].map(mean_meter_reading_per_site_id)
test['median_meter_reading_per_site_id'] = test['site_id'].map(median_meter_reading_per_site_id)
test['std_meter_reading_per_site_id'] = test['site_id'].map(std_meter_reading_per_site_id)

In [ ]:
%%time
for df in [train, test]:
    df['mean_meter_reading_per_building'] = df['mean_meter_reading_per_building'].astype("float16")
    df['median_meter_reading_per_building'] = df['mean_meter_reading_per_building'].astype("float16")
    df['std_meter_reading_per_building'] = df['std_meter_reading_per_building'].astype("float16")
    
    df['mean_meter_reading_on_year_built'] = df['mean_meter_reading_on_year_built'].astype("float16")
    df['median_meter_reading_on_year_built'] = df['median_meter_reading_on_year_built'].astype("float16")
    df['std_meter_reading_on_year_built'] = df['std_meter_reading_on_year_built'].astype("float16")
    
    df['mean_meter_reading_per_meter'] = df['mean_meter_reading_per_meter'].astype("float16")
    df['median_meter_reading_per_meter'] = df['median_meter_reading_per_meter'].astype("float16")
    df['std_meter_reading_per_meter'] = df['std_meter_reading_per_meter'].astype("float16")
    
    df['mean_meter_reading_per_primary_usage'] = df['mean_meter_reading_per_primary_usage'].astype("float16")
    df['median_meter_reading_per_primary_usage'] = df['median_meter_reading_per_primary_usage'].astype("float16")
    df['std_meter_reading_per_primary_usage'] = df['std_meter_reading_per_primary_usage'].astype("float16")
    
    df['mean_meter_reading_per_site_id'] = df['mean_meter_reading_per_site_id'].astype("float16")
    df['median_meter_reading_per_site_id'] = df['median_meter_reading_per_site_id'].astype("float16")
    df['std_meter_reading_per_site_id'] = df['std_meter_reading_per_site_id'].astype("float16")
    
    df['number_unique_meter_per_building'] = df['number_unique_meter_per_building'].astype('uint8')
    df['square_feet'] = df['square_feet'].astype('float16')
gc.collect()

In [ ]:
print (train.shape, test.shape)

In [ ]:
%%time
le = LabelEncoder()

train['meter']= le.fit_transform(train['meter']).astype("uint8")
test['meter']= le.fit_transform(test['meter']).astype("uint8")
train['primary_use']= le.fit_transform(train['primary_use']).astype("uint8")
test['primary_use']= le.fit_transform(test['primary_use']).astype("uint8")

In [ ]:
%%time
# Let's check the correlation between the variables and eliminate the one's that have high correlation
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = train.corr().abs()
corr_matrix.head()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(corr_matrix, annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))
print ("Following columns can be dropped {}".format(to_drop))

In [ ]:
train.drop(to_drop,axis=1,inplace=True)
test.drop(to_drop,axis=1,inplace=True)

In [ ]:
train.corr()['meter_reading'].sort_values()

In [ ]:
train.drop(['air_temperature','precip_depth_1_hr','DayOfMonth','dew_temperature'],axis=1,inplace=True)
test.drop(['air_temperature','precip_depth_1_hr','DayOfMonth','dew_temperature'],axis=1,inplace=True)

In [ ]:
y = train['meter_reading']
train.drop('meter_reading',axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
categorical_cols = ['building_id','Month','meter','Hour','primary_use','DayOfWeek']

일단 여기까지 20191114 04:56